In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("league_record.csv")

In [ ]:
pd.set_option('display.max_columns', None)
df

In [ ]:
# Define function to check home and away scores and return result
def get_result(row):
    if row['HLS'] > row['ALS']:
        return 2  # home_win = 2
    elif row['HLS'] < row['ALS']:
        return 1  # away_win = 1
    else:
        return 0  # draw = 0

In [ ]:
from sklearn.preprocessing import StandardScaler

df_result = df.copy()

# Apply function to each row of the dataframe to create a new column
df_result['result'] = df_result.apply(get_result, axis=1)

df_result = df_result[df_result['week'] > 4]  # remove rows with incomplete values

# remove irrelevant features
df_result = df_result.drop(["league_id", "week", "hour", "minutes", "HLS", "ALS", "HT", "AT", "hl_stat", "al_stat"], axis=1)  

df_result = df_result.reset_index(drop=True)  # reset the index values

# Scaling the dataset using standardization method
X = df_result.drop("result", axis=1)
y = df_result["result"]

# assume that X is your dataset with numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# # concatenate the scaled numerical features with the categorical features
data_preprocessed = pd.concat([pd.DataFrame(X_scaled, columns=X.columns), y], axis=1)

data_preprocessed

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load your dataset into a pandas DataFrame
dataset = data_preprocessed.copy()

# Specify the target variable
target_var = 'result'

# Specify the number of folds for k-fold cross-validation
n_splits = 80

# Initialize the KFold cross-validation object
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize a list to store the accuracy scores for each fold
accuracy_scores = []

# Loop over each fold
for train_idx, test_idx in kfold.split(dataset):
    # Split the data into training and testing sets for the current fold
    X_train, X_test = dataset.drop(target_var, axis=1).iloc[train_idx], dataset.drop(target_var, axis=1).iloc[test_idx]
    y_train, y_test = dataset[target_var].iloc[train_idx], dataset[target_var].iloc[test_idx]
    
    # Initialize a random forest classifier with default hyperparameters
    rf = RandomForestClassifier(random_state=42)
    
    # Fit the classifier to the training data
    rf.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = rf.predict(X_test)
    
    # Calculate the accuracy score for the current fold
    accuracy = accuracy_score(y_test, y_pred)
    
    # Append the accuracy score to the list
    accuracy_scores.append(accuracy)

# Calculate the mean accuracy score across all folds
mean_accuracy = sum(accuracy_scores) / n_splits

print(f"Mean accuracy score: {mean_accuracy}")

In [ ]:
# 4420 Observations
# Mean accuracy score: 0.6326167076167074 - Classifier(Scaled) - 40 splits
# Mean accuracy score: 0.635329417773238- Classifier(Scaled) - 50 splits
# MMean accuracy score: 0.6348327779834632 - Classifier(Scaled) - 60 splits
# Mean accuracy score: 0.6316928854875286 - Classifier(Scaled) - 70 splits
# Mean accuracy score: 0.6337337662337659 - Classifier(Scaled) - 80 splits

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Load your dataset into a pandas DataFrame
dataset = data_preprocessed.copy()
# Specify the target variable
target_var = 'result'

# Specify the number of folds for k-fold cross-validation
n_splits = 50

# Initialize the KFold cross-validation object
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize a list to store the R-squared scores for each fold
r2_scores = []

# Loop over each fold
for train_idx, test_idx in kfold.split(dataset):
    # Split the data into training and testing sets for the current fold
    X_train, X_test = dataset.drop(target_var, axis=1).iloc[train_idx], dataset.drop(target_var, axis=1).iloc[test_idx]
    y_train, y_test = dataset[target_var].iloc[train_idx], dataset[target_var].iloc[test_idx]
    
    # Initialize a random forest regressor with default hyperparameters
    rf = RandomForestRegressor(random_state=42)
    
    # Fit the regressor to the training data
    rf.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = rf.predict(X_test)
    
    # Calculate the R-squared for the current fold
    r2 = r2_score(y_test, y_pred)
    
    # Append the R-squared to the list
    r2_scores.append(r2)

# Calculate the mean R-squared across all folds
mean_r2 = sum(r2_scores) / n_splits

print(f"Mean R-squared: {mean_r2}")

In [ ]:
# 4420 Observations
# Mean R-squared: 0.24548274665914768 - 40 splits
# Mean R-squared: 0.24930608614772112 - Regressor(Scaled) - 50 splits ------->
# Mean R-squared: 0.24661057530469332 - Regressor(Scaled) - 70 splits

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome("chrome_driver/chromedriver.exe")

# Navigate to a web page
driver.get("https://vsmobile.bet9ja.com/bet9ja-mobile/login/?game=league&mode=premier&lang=")
# time.sleep(3)

# menu = driver.find_element(By.CSS_SELECTOR, ".nav")
# hidden_submenu = driver.find_element(By.CSS_SELECTOR, ".nav #submenu1")



# Select all text on the page and copy it to the clipboard
element = driver.find_element(By.XPATH, "//div[@class='ui-panel-wrapper']//i[@class='fa fa-bars icon-menu']")

actions = ActionChains(driver)
# actions.move_to_element(element)
actions.click(element)
actions.perform()

# body.send_keys(Keys.CONTROL + 'a')
# body.send_keys(Keys.CONTROL + 'c')
# Click on the element
# element.click()
# # Get the copied text from the clipboard
# import pyperclip
# copied_text = pyperclip.paste()
 
# # Print the copied text
# print(copied_text)
time.sleep(3)
# Close the browser session
driver.quit()